In [1]:
%matplotlib inline

In [2]:
import warnings
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from copy import deepcopy
from scipy import stats
from scipy.special import comb

import seaborn as sns
import matplotlib.pyplot as plt

from flowanalysis import definitions as fa

import csv
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn3_circles

plt.rcParams.update({"text.usetex": True})
plt.rcParams["text.latex.preamble"] = r"\usepackage{graphicx}"
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [3]:
sns.set_style("ticks")

In [4]:
timeNames = ("Primary", "Memory", "WT challenge", "T8A challege", "N3A challenge")
naiveColumns = ("WT column", "T8A column", "N3A column")
tetramerNames = ("WT", "T8A", "N3A")
naive_stats_column_names = [
    "SP [WT]",
    "T8A [WT]",
    "N3A [WT]",
    "TP [WT]",
    "SP [T8A]",
    "WT [T8A]",
    "N3A [T8A]",
    "TP [T8A]",
    "SP [N3A]",
    "WT [N3A]",
    "T8A [N3A]",
    "TP [N3A]",
]

# Data formating

For each timepoint and sample the data needs to be formated as a 7-tuple, ordered as follows:
1. WT single positive cells
2. T8A single positive cells
3. WT + T8A double positive cells
4. N3A single positive cells
5. WT + N3A double positive cells
6. T8A + N3A double positive cells
7. WT + T8A + N3A triple positive cells

For each priming infection the data is stored on a list indexed as follows:
1. Mouse
2. Timepoint

If there are fewer mice on one of the timepoints the remaining entries on the list must be (-1, 0, 0, 0, 0, 0, 0) to make sure that the mean is correctly calculated

When reading the data the headers are put in a list ordred as follows:
1. Tissue
2. Timepoint
3. WT single positive cells
4. T8A single positive cells
5. WT + T8A double positive cells
6. N3A single positive cells
7. N3A + WT double positive cells
8. N3A + T8A double positive cells
9. WT + T8A + N3A triple positive cells
10. Total CD8 cells

# Spleen circulating

In [5]:
fa.header_clipping("WT", check=True)
fa.header_clipping("T8A", check=True)
fa.header_clipping("N3A", check=True)

['Mouse', 'Cell_Count', 'Tissue', 'Condition', 'Sample:', 'Lymphocytes', 'Singlets', 'Live Cells', 'F480 B220 Negative', 'CD8p_Freq', 'CD8p_Norm_Freq', 'CD8p_Num', 'CD8p_N3ATotal_Freq', 'CD8p_N3ATotal_Num', 'CD8p_T8ATotal_Freq', 'CD8p_T8ATotal_Num', 'CD8p_WTTotal_Freq', 'CD8p_WTTotal_Num', 'CD8p_TriplePos_Freq', 'CD8p_TriplePos_Num', 'CD8p_N3AT8ADP_Freq', 'CD8p_N3AT8ADP_Num', 'CD8p_WTN3ADP_Freq', 'CD8p_WTN3ADP_Num', 'CD8p_WTT8ADP_Freq', 'CD8p_WTT8ADP_Num', 'WTSP_Freq', 'WTSP_Num', 'N3ASP_Freq', 'N3ASP_Num', 'T8ASP_Freq', 'T8ASP_Num']
['Mouse', 'Cell Count', 'Tissue', 'T8A Primary', 'Sample:', 'Lymphocytes', 'Singlets', 'Live Cells', 'F480 B220 Negative', 'CD8p_Freq', 'CD8p_Norm_Freq', 'CD8p_Num', 'CD8p_N3ATotal_Freq', 'CD8p_N3ATotal_Num', 'CD8p_T8ATotal_Freq', 'CD8p_T8ATotal_Num', 'CD8p_WTTotal_Freq', 'CD8p_WTTotal_Num', 'CD8p_TriplePos_Freq', 'CD8p_TriplePos_Num', 'CD8p_N3AT8ADP_Freq', 'CD8p_N3AT8ADP_Num', 'CD8p_WTN3ADP_Freq', 'CD8p_WTN3ADP_Num', 'CD8p_WTT8ADP_Freq', 'CD8p_WTT8ADP_Num

In [6]:
fa.header_clipping("WT")
fa.header_clipping("T8A")
fa.header_clipping("N3A")

wt_p_headers = [
    "Tissue",
    "Condition",
    "WTSP_Num",
    "T8ASP_Num",
    "CD8p_WTT8ADP_Num",
    "N3ASP_Num",
    "CD8p_WTN3ADP_Num",
    "CD8p_N3AT8ADP_Num",
    "CD8p_TriplePos_Num",
    "CD8p_Num",
]
t8a_p_headers = [
    "Tissue",
    "T8A Primary",
    "CD8p_WTSP_Num",
    "CD8p_T8ASP_Num",
    "CD8p_WTT8ADP_Num",
    "CD8p_N3ASP_Num",
    "CD8p_WTN3ADP_Num",
    "CD8p_N3AT8ADP_Num",
    "CD8p_TriplePos_Num",
    "CD8p_Num",
]
n3a_p_headers = [
    "Tissue",
    "Infection",
    "CD8p_WTSP_Num",
    "CD8p_T8ASP_Num",
    "CD8p_WTT8ADP_Num",
    "CD8p_N3ASP_Num",
    "CD8p_N3AWTDP_Num",
    "CD8p_N3AT8ADP_Num",
    "CD8p_TriplePos_Num",
    "CD8+CD45-_Norm_Num",
]

wt_p_time_names = fa.time_name_list("WT", wt_p_headers)
t8a_p_time_names = fa.time_name_list("T8A", t8a_p_headers)
n3a_p_time_names = fa.time_name_list("N3A", n3a_p_headers)

wt_p_data, wt_p_data_neg = fa.data_extraction(
    "WT", "spleen", wt_p_headers, wt_p_time_names, timepoints=5
)
t8a_p_data, t8a_p_data_neg = fa.data_extraction(
    "T8A", "spleen", t8a_p_headers, t8a_p_time_names, timepoints=5
)
n3a_p_data, n3a_p_data_neg = fa.data_extraction(
    "N3A", "spleen", n3a_p_headers, n3a_p_time_names, timepoints=5
)

In [7]:
fa.venn_plots(
    5,
    wt_p_data,
    timeNames,
    tetramerNames,
    title="Spleen Circulating - WT primary",
    file_name="WT-C",
)

/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:61: UserWarning: Circle C has zero area
  warnings.warn("Circle C has zero area")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:117: UserWarning: Bad circle positioning
  warnings.warn("Bad circle positioning")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:57: UserWarning: Circle B has zero area
  warnings.warn("Circle B has zero area")


In [8]:
fa.venn_plots(
    5,
    t8a_p_data,
    timeNames,
    tetramerNames,
    title="Spleen Circulating - T8A primary",
    file_name="T8A-C",
)

/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:53: UserWarning: Circle A has zero area
  warnings.warn("Circle A has zero area")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:61: UserWarning: Circle C has zero area
  warnings.warn("Circle C has zero area")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:117: UserWarning: Bad circle positioning
  warnings.warn("Bad circle positioning")


In [9]:
fa.venn_plots(
    5,
    n3a_p_data,
    timeNames,
    tetramerNames,
    title="Spleen Circulating - N3A primary",
    file_name="N3A-C",
)

/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:57: UserWarning: Circle B has zero area
  warnings.warn("Circle B has zero area")


# Slopes

## Circulating

In [10]:
wt_p_means, wt_p_actual_mice = fa.population_means(wt_p_data)
t8a_p_means, t8a_p_actual_mice = fa.population_means(t8a_p_data)
n3a_p_means, n3a_p_actual_mice = fa.population_means(n3a_p_data)

wt_p_means_neg, wt_p_actual_mice_neg = fa.population_means(wt_p_data_neg)
t8a_p_means_neg, t8a_p_actual_mice_neg = fa.population_means(t8a_p_data_neg)
n3a_p_means_neg, n3a_p_actual_mice_neg = fa.population_means(n3a_p_data_neg)

patch_names = [
    "WT",
    "T8A",
    "N3A",
    "WT+T8A",
    "WT+N3A",
    "T8A+N3A",
    "Triple positive",
    "Triple negative",
]
patch_indices = [0, 1, 3, 2, 4, 5, 6]
times = [10, 70, 90]

In [11]:
fa.slope_plots(
    "Spleen circulating - WT primary",
    wt_p_means,
    wt_p_means_neg,
    times,
    patch_names,
    patch_indices,
    file_name="WT-C-S",
    zeroline=True,
)

In [12]:
fa.slope_plots(
    "Spleen circulating - T8A primary",
    t8a_p_means,
    t8a_p_means_neg,
    times,
    patch_names,
    patch_indices,
    file_name="T8A-C-S",
    zeroline=True,
)

In [13]:
fa.slope_plots(
    "Spleen circulating - N3A primary",
    n3a_p_means,
    n3a_p_means_neg,
    times,
    patch_names,
    patch_indices,
    file_name="N3A-C-S",
    zeroline=True,
)

## Resident

In [14]:
fa.header_clipping("WT", cd45="-", check=True)
fa.header_clipping("T8A", cd45="-", check=True)
fa.header_clipping("N3A", cd45="-", check=True)

['Mouse', 'Cell Count', 'Tissue', 'Condition', 'Sample:', 'Lymphocytes', 'Singlets', 'Live Cells', 'F480 B220 Negative', 'CD8n_Freq', 'CD8n_Norm_Freq', 'CD8n_Num', 'CD8n_N3ATotal_Freq', 'CD8n_N3ATotal_Num', 'CD8n_T8ATotal_Freq', 'CD8n_T8Atotal_Num', 'CD8n_WTTotal_Freq', 'CD8n_WTTotal_Num', 'CD8n_Triplepos_Freq', 'CD8n_Triplepos_Num', 'CD8n_N3AT8ADP_Freq', 'CD8n_N3AT8ADP_Num', 'CD8n_WTN3ADP_Freq', 'CD8n_WTN3ADP_Num', 'CD8n_WTT8ADP_Freq', 'CD8n_WTT8ADP_Num', 'CD8n_WTSP_Freq', 'CD8n_WTSP_Num', 'CD8n_N3ASP_Freq', 'CD8n_N3ASP_Num', 'CD8n_T8ASP_Freq', 'CD8n_T8ASP_Num', 'Triple Neg', 'total frequency tet+ cells']
['Mouse', 'Cell Count', 'Tissue', 'Infection', 'Sample:', 'Lymphocytes', 'Singlets', 'Live Cells', 'F480 B220 Negative', 'CD8n_Freq', 'CD8n_Norm_Freq', 'CD8n_Num', 'CD8n_N3A_Total_Freq', 'CD8n_N3A_Total_Num', 'CD8n_T8ATotalFreq', 'CD8p_T8ATotal_Num', 'CD8p_WTTotal_Freq', 'CD8p_WTTotal_Num', 'CD8p_TriplePos_Freq', 'CD8p_TriplePos_Num', 'CD8p_N3AT8ADP_Freq', 'CD8p_N3AT8ADP_Num', 'CD8p_

In [15]:
fa.header_clipping("WT", cd45="-")
fa.header_clipping("T8A", cd45="-")
fa.header_clipping("N3A", cd45="-")

wt_n_headers = [
    "Tissue",
    "Condition",
    "CD8n_WTSP_Num",
    "CD8n_T8ASP_Num",
    "CD8n_WTT8ADP_Num",
    "CD8n_N3ASP_Num",
    "CD8n_WTN3ADP_Num",
    "CD8n_N3AT8ADP_Num",
    "CD8n_Triplepos_Num",
    "CD8n_Num",
]
t8a_n_headers = [
    "Tissue",
    "Infection",
    "CD8p_WTSP_Num",
    "CD8p_T8ASP_Num",
    "CD8p_WTT8ADP_Num",
    "CD8p_N3ASP_Num",
    "CD8p_WTN3ADP_Num",
    "CD8p_N3AT8ADP_Num",
    "CD8p_TriplePos_Num",
    "CD8n_Num",
]
n3a_n_headers = [
    "Tissue",
    "Infection",
    "CD8n_WTSP_Num",
    "CD8n_T8ASP_Num",
    "CD8n_WTT8ADP_Num",
    "CD8n_N3ASP_Num",
    "CD8n_N3AWTDP_Num",
    "CD8n_N3AT8ADP_Num",
    "CD8n_TriplePos_Num",
    "CD8+CD45-_Norm_Num",
]

wt_n_time_names = fa.time_name_list("WT", wt_n_headers, cd45="-")
t8a_n_time_names = fa.time_name_list("T8A", t8a_n_headers, cd45="-")
n3a_n_time_names = fa.time_name_list("N3A", n3a_n_headers, cd45="-")

wt_n_data, wt_n_data_neg = fa.data_extraction(
    "WT", "spleen", wt_n_headers, wt_n_time_names, cd45="-", timepoints=5
)
t8a_n_data, t8a_n_data_neg = fa.data_extraction(
    "T8A", "spleen", t8a_n_headers, t8a_n_time_names, cd45="-", timepoints=5
)
n3a_n_data, n3a_n_data_neg = fa.data_extraction(
    "N3A", "spleen", n3a_n_headers, n3a_n_time_names, cd45="-", timepoints=5
)

wt_n_means, wt_n_actual_mice = fa.population_means(wt_n_data)
t8a_n_means, t8a_n_actual_mice = fa.population_means(t8a_n_data)
n3a_n_means, n3a_n_actual_mice = fa.population_means(n3a_n_data)

wt_n_means_neg, wt_n_actual_mice_neg = fa.population_means(wt_n_data_neg)
t8a_n_means_neg, t8a_n_actual_mice_neg = fa.population_means(t8a_n_data_neg)
n3a_n_means_neg, n3a_n_actual_mice_neg = fa.population_means(n3a_n_data_neg)

In [16]:
fa.slope_plots(
    "Spleen resident - WT primary",
    wt_n_means,
    wt_n_means_neg,
    times,
    patch_names,
    patch_indices,
    file_name="WT-R-S",
    zeroline=True,
)

In [17]:
fa.slope_plots(
    "Spleen resident - T8A primary",
    t8a_n_means,
    t8a_n_means_neg,
    times,
    patch_names,
    patch_indices,
    file_name="T8A-R-S",
    zeroline=True,
)

In [18]:
fa.slope_plots(
    "Spleen resident - N3A primary",
    n3a_n_means,
    n3a_n_means_neg,
    times,
    patch_names,
    patch_indices,
    file_name="N3A-R-S",
    zeroline=True,
)

In [19]:
fa.venn_plots(
    5,
    wt_n_data,
    timeNames,
    tetramerNames,
    title="Spleen resident - WT primary",
    file_name="WT-R",
)

In [20]:
fa.venn_plots(
    5,
    t8a_n_data,
    timeNames,
    tetramerNames,
    title="Spleen resident - T8A primary",
    file_name="T8A-R",
)

In [21]:
fa.venn_plots(
    5,
    n3a_n_data,
    timeNames,
    tetramerNames,
    title="Spleen resident - N3A primary",
    file_name="N3A-R",
)

# Stats

## Challenge timepoint WT

### Loading data

In [22]:
wt_chDF = fa.stats_dataframe_infection(
    (wt_p_data, wt_p_data_neg),
    ["Challenge", "WT", "T8A", "WT+T8A", "N3A", "WT+N3A", "T8A+N3A", "TP", "TN"],
    complete=True,
)

wt_chDF.to_csv("WT.csv", index=False)

In [23]:
wt_chDF_freq = deepcopy(wt_chDF)
int_columns = ["WT", "T8A", "WT+T8A", "N3A", "WT+N3A", "T8A+N3A", "TP", "TN"]

wt_chDF_freq[int_columns] = wt_chDF_freq[int_columns].div(
    wt_chDF_freq[int_columns].sum(axis=1), axis=0
)

wt_chDF_freq.to_csv("WT-f.csv", index=False)

# wt_chDF_freq

### Combined plots

In [24]:
grid = sns.PairGrid(
    data=wt_chDF[(wt_chDF.Challenge != "Primary") & (wt_chDF.Challenge != "Memory")],
    hue="Challenge",
    hue_kws={"corr_position": [0, 1, 2]},
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2), corr_total=3)

for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")
# grid.tight_layout()

grid.add_legend()
# grid.add_legend(fontsize=20, title_fontproperties={"size": 25})

grid.figure.savefig("Challenge-stats.pdf")
plt.close("all")

In [25]:
grid = sns.PairGrid(
    data=wt_chDF_freq[
        (wt_chDF_freq.Challenge != "Primary") & (wt_chDF_freq.Challenge != "Memory")
    ],
    hue="Challenge",
    hue_kws={"corr_position": [0, 1, 2]},
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2), corr_total=3)

for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")
# grid.tight_layout()

grid.add_legend()
# grid.add_legend(fontsize=20, title_fontproperties={"size": 25})

grid.figure.savefig("Challenge-stats-freq.pdf")
plt.close("all")

In [26]:
grid = sns.PairGrid(
    data=wt_chDF[(wt_chDF.Challenge != "Primary") & (wt_chDF.Challenge != "Memory")],
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

grid.figure.savefig("Challenge-stats-combined.pdf")
plt.close("all")

In [27]:
grid = sns.PairGrid(
    data=wt_chDF_freq[
        (wt_chDF_freq.Challenge != "Primary") & (wt_chDF_freq.Challenge != "Memory")
    ],
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

grid.figure.savefig("Challenge-stats-combined-freq.pdf")
plt.close("all")

### Single plots

In [28]:
grid = sns.PairGrid(
    data=wt_chDF[(wt_chDF.Challenge == "WT")], height=1.5, diag_sharey=False
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(7, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

# grid.figure.savefig("Challenge-stats-WT.pdf")
plt.close("all")

In [29]:
grid = sns.PairGrid(
    data=wt_chDF_freq[(wt_chDF_freq.Challenge == "WT")], height=1.5, diag_sharey=False
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(7, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

# grid.figure.savefig("Challenge-stats-WT-freq.pdf")
plt.close("all")

In [30]:
grid = sns.PairGrid(
    data=wt_chDF[(wt_chDF.Challenge == "N3A")], height=1.5, diag_sharey=False
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(7, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

# grid.figure.savefig("Challenge-stats-N3A.pdf")
plt.close("all")

In [31]:
grid = sns.PairGrid(
    data=wt_chDF_freq[(wt_chDF_freq.Challenge == "N3A")], height=1.5, diag_sharey=False
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(7, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

# grid.figure.savefig("Challenge-stats-N3A-freq.pdf")
plt.close("all")

In [32]:
grid = sns.PairGrid(
    data=wt_chDF[(wt_chDF.Challenge == "T8A")], height=1.5, diag_sharey=False
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

# grid.figure.savefig("Challenge-stats-T8A.pdf")
plt.close("all")

In [33]:
grid = sns.PairGrid(
    data=wt_chDF_freq[(wt_chDF_freq.Challenge == "T8A")], height=1.5, diag_sharey=False
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

# grid.figure.savefig("Challenge-stats-T8A-freq.pdf")
plt.close("all")

In [34]:
wt_chDF[(wt_chDF_freq.Challenge == "T8A")]

,Challenge,WT,T8A,WT+T8A,N3A,WT+N3A,T8A+N3A,TP,TN
3,T8A,10500,879000,475000,2150,2150,1050,40600,4739550
8,T8A,10800,26600,136000,1210,4760,0,41200,2949430
13,T8A,242000,257000,932000,8310,9740,4160,242000,10204790
18,T8A,27000,314000,70100,5930,2370,0,1190,10379410
21,T8A,22800,248000,277000,3610,3610,2360,288000,7014620
22,T8A,27300,464000,224000,0,12500,0,15600,7046600


## Challenge timepoint T8A

### Loading data

In [35]:
t8a_chDF = fa.stats_dataframe_infection(
    (t8a_p_data, t8a_p_data_neg),
    ["Challenge", "WT", "T8A", "WT+T8A", "N3A", "WT+N3A", "T8A+N3A", "TP", "TN"],
    complete=True,
)

t8a_chDF_freq = deepcopy(t8a_chDF)
int_columns = ["WT", "T8A", "WT+T8A", "N3A", "WT+N3A", "T8A+N3A", "TP", "TN"]

t8a_chDF_freq[int_columns] = t8a_chDF_freq[int_columns].div(
    t8a_chDF_freq[int_columns].sum(axis=1), axis=0
)

t8a_chDF.to_csv("T8A.csv", index=False)
t8a_chDF_freq.to_csv("T8A-f.csv", index=False)

### Combined plots

In [36]:
grid = sns.PairGrid(
    data=t8a_chDF[(t8a_chDF.Challenge != "Primary") & (t8a_chDF.Challenge != "Memory")],
    hue="Challenge",
    hue_kws={"corr_position": [0, 1, 2]},
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2), corr_total=3)

for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")
# grid.tight_layout()

grid.add_legend()
# grid.add_legend(fontsize=20, title_fontproperties={"size": 25})

grid.figure.savefig("Challenge-stats[T8A].pdf")
plt.close("all")

In [37]:
grid = sns.PairGrid(
    data=t8a_chDF_freq[
        (t8a_chDF_freq.Challenge != "Primary") & (t8a_chDF_freq.Challenge != "Memory")
    ],
    hue="Challenge",
    hue_kws={"corr_position": [0, 1, 2]},
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2), corr_total=3)

for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")
# grid.tight_layout()

grid.add_legend()
# grid.add_legend(fontsize=20, title_fontproperties={"size": 25})

grid.figure.savefig("Challenge-stats-freq[T8A].pdf")
plt.close("all")

In [38]:
grid = sns.PairGrid(
    data=t8a_chDF[(t8a_chDF.Challenge != "Primary") & (t8a_chDF.Challenge != "Memory")],
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

grid.figure.savefig("Challenge-stats-combined[T8A].pdf")
plt.close("all")

In [39]:
grid = sns.PairGrid(
    data=t8a_chDF_freq[
        (t8a_chDF_freq.Challenge != "Primary") & (t8a_chDF_freq.Challenge != "Memory")
    ],
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

grid.figure.savefig("Challenge-stats-combined-freq[T8A].pdf")
plt.close("all")

### Single plots

## Challenge timepoint N3A

### Loading data

In [40]:
n3a_chDF = fa.stats_dataframe_infection(
    (n3a_p_data, n3a_p_data_neg),
    ["Challenge", "WT", "T8A", "WT+T8A", "N3A", "WT+N3A", "T8A+N3A", "TP", "TN"],
    complete=True,
)

n3a_chDF_freq = deepcopy(n3a_chDF)
int_columns = ["WT", "T8A", "WT+T8A", "N3A", "WT+N3A", "T8A+N3A", "TP", "TN"]

n3a_chDF_freq[int_columns] = n3a_chDF_freq[int_columns].div(
    n3a_chDF_freq[int_columns].sum(axis=1), axis=0
)

n3a_chDF.to_csv("N3A.csv", index=False)
n3a_chDF_freq.to_csv("N3A-f.csv", index=False)

### Combined plots

In [41]:
grid = sns.PairGrid(
    data=n3a_chDF[(n3a_chDF.Challenge != "Primary") & (n3a_chDF.Challenge != "Memory")],
    hue="Challenge",
    hue_kws={"corr_position": [0, 1, 2]},
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2), corr_total=3)

for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")
# grid.tight_layout()

grid.add_legend()
# grid.add_legend(fontsize=20, title_fontproperties={"size": 25})

grid.figure.savefig("Challenge-stats[N3A].pdf")
plt.close("all")

In [42]:
grid = sns.PairGrid(
    data=n3a_chDF_freq[
        (n3a_chDF_freq.Challenge != "Primary") & (n3a_chDF_freq.Challenge != "Memory")
    ],
    hue="Challenge",
    hue_kws={"corr_position": [0, 1, 2]},
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2), corr_total=3)

for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")
# grid.tight_layout()

grid.add_legend()
# grid.add_legend(fontsize=20, title_fontproperties={"size": 25})

grid.figure.savefig("Challenge-stats-freq[N3A].pdf")
plt.close("all")

In [43]:
grid = sns.PairGrid(
    data=n3a_chDF[(n3a_chDF.Challenge != "Primary") & (n3a_chDF.Challenge != "Memory")],
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

grid.figure.savefig("Challenge-stats-combined[N3A].pdf")
plt.close("all")

In [44]:
grid = sns.PairGrid(
    data=n3a_chDF_freq[
        (n3a_chDF_freq.Challenge != "Primary") & (n3a_chDF_freq.Challenge != "Memory")
    ],
    height=1.5,
    diag_sharey=False,
)
grid.map_diag(sns.kdeplot, warn_singular=False)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(8, 2))
for ax in grid.axes.flatten():
    ax.ticklabel_format(style="sci", scilimits=(0, 0), axis="both")

grid.figure.savefig("Challenge-stats-combined-freq[N3A].pdf")
plt.close("all")

### Single plots

## Naive

### Absolute numbers

In [45]:
fa.header_clipping("Naive", check=True)

['Mouse', 'Tissue', 'Cell Count', 'Experiment', 'Stain for Column Selection', 'Sample:', 'Lymphocytes', 'Singlets', 'Live Cells', 'B220 F480 Neg', 'CD8_Freq', 'CD8_Num', 'CD8_N3ATotal_Freq', 'CD8_N3ATotal_Num', 'CD8_T8ATotal_Freq', 'CD8_T8ATotal_Num', 'CD8_WTTotal_Freq', 'CD8_WTTotal_Num', 'N3A Neg', 'T8A Neg', 'WT Neg', 'CD8_TriplePos_Freq', 'CD8_TriplePos_Num', 'CD8_N3AT8ADP_Freq', 'CD8_N3AT8ADP_Num', 'CD8_WTN3ADP_Freq', 'CD8_WTN3ADP_Num', 'CD8_N3ASP_Freq', 'CD8_N3ASP_Num', 'CD8_WTT8ADP_Freq', 'CD8_WTT8ADP_Num', 'CD8_T8ASP_Freq', 'CD8_T8ASP_Num', 'CD8_WTSP_Freq', 'CD8_WTSP_Num', 'CD8_TripleNeg_Freq', 'CD8_TripleNeg_Num']


In [46]:
fa.header_clipping("Naive")

naive_headers = [
    "Tissue",
    "Stain for Column Selection",
    "CD8_WTSP_Num",
    "CD8_T8ASP_Num",
    "CD8_WTT8ADP_Num",
    "CD8_N3ASP_Num",
    "CD8_WTN3ADP_Num",
    "CD8_N3AT8ADP_Num",
    "CD8_TriplePos_Num",
    "CD8_Num",
]

naive_column_names = fa.time_name_list("Naive", naive_headers)

naive_data, naive_data_neg = fa.data_extraction(
    "Naive", "Spleen + Lymphnodes", naive_headers, naive_column_names
)

naive_means, naive_actual_mice = fa.population_means(naive_data)

naive_means_neg, naive_actual_mice_neg = fa.population_means(naive_data_neg)

In [47]:
fa.venn_plots(
    3,
    naive_data,
    naiveColumns,
    tetramerNames,
    title="Naive data",
    file_name="Naive-Venn",
    # ignore=[0, 4],
)

/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:117: UserWarning: Bad circle positioning
  warnings.warn("Bad circle positioning")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:53: UserWarning: Circle A has zero area
  warnings.warn("Circle A has zero area")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:61: UserWarning: Circle C has zero area
  warnings.warn("Circle C has zero area")


In [48]:
fa.header_clipping("Naive", filename="WT", check=True)
fa.header_clipping("Naive", filename="T8A", check=True)
fa.header_clipping("Naive", filename="N3A", check=True)

['Mouse', 'Tissue', 'Cell Count', 'Experiment', 'Stain for Column Selection', 'Sample:', 'Lymphocytes', 'Singlets', 'Live Cells', 'B220 F480 Neg', 'CD8_Freq', 'CD8_Num', 'Column Selected WT Positive_Freq', 'Column Selected WT Positive_Num', 'CD8_WT_N3ATotal_Freq', 'CD8_WT_N3ATotal_Num', 'CD8_WT_T8ATotal_Freq', 'CD8_WT_T8ATotal_Num', 'CD8_WT_N3Aneg_Freq', 'CD8_WT_N3Aneg_Num', 'CD8_WT_T8Aneg_Freq', 'CD8_WT_T8Aneg_Num', 'CD8_WT_N3AT8ADP_Freq', 'CD8_WT_N3AT8ADP_Num', 'CD8_WT_N3ASP_Freq', 'CD8_WT_N3ASP_Num', 'CD8_WT_T8ASP_Freq', 'CD8_WT_T8ASP_Num', 'CD8_WT_N3AT8Aneg_Freq', 'CD8_WT_N3AT8Aneg_Num']
['Mouse', 'Tissue', 'Cell Count', 'Experiment', 'Stain for Column Selection', 'Sample:', 'Lymphocytes', 'Singlets', 'Live Cells', 'B220 F480 Neg', 'CD8_Freq', 'CD8_Num', 'Column Selected T8A Positive_Freq', 'Column Selected T8A Positive_Num', 'CD8_T8A_N3ATotal_Freq', 'CD8_T8A_N3ATotal_Num', 'CD8_T8A_WTTotal_Freq', 'CD8_T8A_WTTotal_Num', 'CD8_T8A_N3Aneg_Freq', 'CD8_T8A_N3Aneg_Num', 'CD8_T8A_WTneg_Fr

In [49]:
fa.header_clipping("Naive", filename="WT")
fa.header_clipping("Naive", filename="T8A")
fa.header_clipping("Naive", filename="N3A")

naive_wt_headers = [
    "Tissue",
    "Stain for Column Selection",
    "CD8_WT_N3AT8Aneg_Num",
    "CD8_WT_T8ASP_Num",
    "CD8_WT_N3ASP_Num",
    "CD8_WT_N3AT8ADP_Num",
    "Column Selected WT Positive_Num",
]

naive_t8a_headers = [
    "Tissue",
    "Stain for Column Selection",
    "CD8_T8A_N3AWTneg_Num",
    "CD8_T8A_WTSP_Num",
    "CD8_T8A_N3ASP_Num",
    "CD8_T8A_N3AWTDP_Num",
    "Column Selected T8A Positive_Num",
]

naive_n3a_headers = [
    "Tissue",
    "Stain for Column Selection",
    "CD8_N3A_WTT8Aneg_Num",
    "CD8_N3A_WTSP_Num",
    "CD8_N3A_T8ASP_Num",
    "CD8_N3A_T8AWTDP_Num",
    "Column Selected N3A Positive_Num",
]

naive_wt_column_names = fa.time_name_list("Naive", naive_wt_headers, filename="WT")
naive_t8a_column_names = fa.time_name_list("Naive", naive_t8a_headers, filename="T8A")
naive_n3a_column_names = fa.time_name_list("Naive", naive_n3a_headers, filename="N3A")

naive_wt_data, _ = fa.data_extraction(
    "Naive",
    "Spleen + Lymphnodes",
    naive_wt_headers,
    naive_wt_column_names,
    data_type="naive",
    column="WT",
    filename="WT",
)

naive_t8a_data, _ = fa.data_extraction(
    "Naive",
    "Spleen + Lymphnodes",
    naive_t8a_headers,
    naive_t8a_column_names,
    data_type="naive",
    column="T8A",
    filename="T8A",
)

naive_n3a_data, _ = fa.data_extraction(
    "Naive",
    "Spleen + Lymphnodes",
    naive_n3a_headers,
    naive_n3a_column_names,
    data_type="naive",
    column="N3A",
    filename="N3A",
)

In [50]:
combined_naive_data = fa.combine_naive_data(
    [naive_wt_data, naive_t8a_data, naive_n3a_data]
)

In [51]:
statsDF = fa.stats_dataframe_naive(combined_naive_data, naive_stats_column_names)

In [52]:
# sns.pairplot(statsDF, diag_kind="kde", height=1.5)

grid = sns.PairGrid(data=statsDF, height=1.5, diag_sharey=False)
grid.map_diag(sns.kdeplot)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(12, 2))
grid.figure.savefig("Naive-stats.pdf")
# grid.tight_layout()

plt.close("all")

In [53]:
fa.venn_plots(
    3,
    combined_naive_data,
    naiveColumns,
    tetramerNames,
    title="Naive data",
    file_name="Naive",
)

/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:117: UserWarning: Bad circle positioning
  warnings.warn("Bad circle positioning")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:53: UserWarning: Circle A has zero area
  warnings.warn("Circle A has zero area")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:61: UserWarning: Circle C has zero area
  warnings.warn("Circle C has zero area")


### Frequencies

In [54]:
fa.header_clipping("Naive", filename="WT")
fa.header_clipping("Naive", filename="T8A")
fa.header_clipping("Naive", filename="N3A")

naive_wt_headers = [
    "Tissue",
    "Stain for Column Selection",
    "CD8_WT_N3AT8Aneg_Num",
    "CD8_WT_T8ASP_Num",
    "CD8_WT_N3ASP_Num",
    "CD8_WT_N3AT8ADP_Num",
    "Column Selected WT Positive_Num",
]

naive_t8a_headers = [
    "Tissue",
    "Stain for Column Selection",
    "CD8_T8A_N3AWTneg_Num",
    "CD8_T8A_WTSP_Num",
    "CD8_T8A_N3ASP_Num",
    "CD8_T8A_N3AWTDP_Num",
    "Column Selected T8A Positive_Num",
]

naive_n3a_headers = [
    "Tissue",
    "Stain for Column Selection",
    "CD8_N3A_WTT8Aneg_Num",
    "CD8_N3A_WTSP_Num",
    "CD8_N3A_T8ASP_Num",
    "CD8_N3A_T8AWTDP_Num",
    "Column Selected N3A Positive_Num",
]

naive_wt_column_names = fa.time_name_list("Naive", naive_wt_headers, filename="WT")
naive_t8a_column_names = fa.time_name_list("Naive", naive_t8a_headers, filename="T8A")
naive_n3a_column_names = fa.time_name_list("Naive", naive_n3a_headers, filename="N3A")

naive_wt_data_freq, naive_wt_data_neg_norm = fa.data_extraction(
    "Naive",
    "Spleen + Lymphnodes",
    naive_wt_headers,
    naive_wt_column_names,
    data_type="naive",
    column="WT",
    filename="WT",
    normalise=True,
)

naive_t8a_data_freq, naive_t8a_data_neg_norm = fa.data_extraction(
    "Naive",
    "Spleen + Lymphnodes",
    naive_t8a_headers,
    naive_t8a_column_names,
    data_type="naive",
    column="T8A",
    filename="T8A",
    normalise=True,
)

naive_n3a_data_freq, naive_n3a_data_neg_norm = fa.data_extraction(
    "Naive",
    "Spleen + Lymphnodes",
    naive_n3a_headers,
    naive_n3a_column_names,
    data_type="naive",
    column="N3A",
    filename="N3A",
    normalise=True,
)

combined_naive_data_freq = fa.combine_naive_data(
    [naive_wt_data_freq, naive_t8a_data_freq, naive_n3a_data_freq]
)

stats_freqDF = fa.stats_dataframe_naive(
    combined_naive_data_freq, naive_stats_column_names
)

In [55]:
grid = sns.PairGrid(data=stats_freqDF, height=1.5, diag_sharey=False)
grid.map_diag(sns.kdeplot)
grid.map_lower(sns.scatterplot)
grid.map_upper(fa.pairs_stats, comparisons=comb(12, 2))
grid.figure.savefig("Naive-freq-stats.pdf")

plt.close("all")

In [56]:
fa.venn_plots(
    3,
    combined_naive_data_freq,
    naiveColumns,
    tetramerNames,
    title="Naive data (frequencies)",
    file_name="Naive-freq",
    decimals=3,
)

/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:117: UserWarning: Bad circle positioning
  warnings.warn("Bad circle positioning")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:53: UserWarning: Circle A has zero area
  warnings.warn("Circle A has zero area")
/Users/danny/anaconda3/envs/P37T2/lib/python3.7/site-packages/matplotlib_venn/_venn3.py:61: UserWarning: Circle C has zero area
  warnings.warn("Circle C has zero area")


# Plotly tests

## Plots

In [57]:
data = [
    wt_p_data,
    t8a_p_data,
    n3a_p_data,
    wt_p_data_neg,
    t8a_p_data_neg,
    n3a_p_data_neg,
]
priming = ["WT", "T8A", "N3A"]
time = [10, 70, 90, 90, 90]
organ = ["Spleen", "Spleen", "Spleen"]
cd45 = ["-", "-", "-"]
columns = ["Organ", "CD45", "Priming", "Time (dpi)", "Cells", "Tetramer", "Timepoint"]
infection_dict = {0: "Primary", 1: "Memory", 2: "WT", 3: "T8A", 4: "N3A"}
patch_dict = {
    0: "WT",
    1: "T8A",
    3: "N3A",
    2: "WT+T8A",
    4: "WT+N3A",
    5: "T8A+N3A",
    6: "Triple positive",
}

dataDF = fa.plot_dataframe(
    data, priming, time, organ, cd45, columns, patch_dict, infection_dict
)

In [58]:
fig = px.line(
    dataDF[
        (dataDF.Timepoint == "Primary")
        | (dataDF.Timepoint == "Memory")
        | (dataDF.Timepoint == "WT")
    ],
    x="Time (dpi)",
    y="Cells",
    color="Tetramer",
    title="Contration and expansion slopes",
    markers=True,
    log_y=False  # ,
    # symbol='Priming'
)

# fig.show()

fig.write_html("single-slope-plot.html")

In [59]:
dataDF[
    (dataDF.Timepoint == "WT")
    | (dataDF.Timepoint == "T8A")
    | (dataDF.Timepoint == "N3A")
]

,Organ,CD45,Priming,Time (dpi),Cells,Tetramer,Timepoint
14,Spleen,-,WT,90,1057312.00,WT,WT
15,Spleen,-,WT,90,32473.25,T8A,WT
16,Spleen,-,WT,90,1130300.00,WT+T8A,WT
17,Spleen,-,WT,90,1815.00,N3A,WT
18,Spleen,-,WT,90,11830.00,WT+N3A,WT
...,...,...,...,...,...,...,...
113,Spleen,-,T8A,90,11359368.00,Total,T8A
114,Spleen,-,T8A,90,7176572.20,Total,N3A
117,Spleen,-,N3A,90,17245409.67,Total,WT
118,Spleen,-,N3A,90,4330925.75,Total,T8A


## Multi-col test

In [60]:
times = [10, 70, 90]
left_col = [0, 1, 2, 6, 7, 8, 12, 13, 14]
right_col = [3, 4, 5, 9, 10, 11, 15, 16, 17]
colours = ["#29e0c3", "#f28b2e", "#34a0e0"]
tetramers = ["WT", "T8A", "N3A"]

fig = go.Figure().set_subplots(
    rows=3,
    cols=2,
    subplot_titles=(
        "WT primary",
        "WT primary",
        "T8A primary",
        "T8A primary",
        "N3A primary",
        "N3A primary",
    ),
    shared_xaxes=True,
    shared_yaxes=True,
    vertical_spacing=0.05,
)

fig.update_layout(
    legend_title_text="Challenge infection",
    legend=dict(
        yanchor="bottom",
        xanchor="center",
        y=1.02,
        x=0.5,
        orientation="h",
        font={"size": 18},
    ),
    height=1200,
    width=750,
)

fig.update_xaxes(title_text="Time (dpi)", row=3, col=1)
fig.update_xaxes(title_text="Time (dpi)", row=3, col=2)
fig.update_xaxes(tickvals=[10, 70, 90], title_font={"size": 14})
fig.update_yaxes(title_text="Cells", title_font={"size": 14}, col=1)
# fig.update_yaxes(type='log')
# , rangemode='nonnegative')


# WT primary

# Left column
for colour, challenge in enumerate(tetramers):
    fig.add_trace(
        go.Scatter(
            x=times,
            y=fa.separate_data(dataDF, "WT", challenge, "WT").Cells,
            mode="lines+markers",
            name=challenge,
            line=dict(color=colours[colour]),
        ),
        1,
        1,
    )


# Right column
for colour, challenge in enumerate(tetramers):
    fig.add_trace(
        go.Scatter(
            x=times,
            y=fa.separate_data(dataDF, "WT", challenge, "WT").Cells,
            mode="lines+markers",
            name=challenge,
            showlegend=False,
            line=dict(color=colours[colour]),
        ),
        1,
        2,
    )

# SEPARATE INTO ONE LINE FOR FIRST CONTRACTION AND 3 LINES FOR EXPASIONS


# T8A primary

# Left column
for colour, challenge in enumerate(tetramers):
    fig.add_trace(
        go.Scatter(
            x=times,
            y=fa.separate_data(dataDF, "T8A", challenge, "WT").Cells,
            mode="lines+markers",
            name=challenge,
            showlegend=False,
            line=dict(color=colours[colour]),
        ),
        2,
        1,
    )

# Right column
for colour, challenge in enumerate(tetramers):
    fig.add_trace(
        go.Scatter(
            x=times,
            y=fa.separate_data(dataDF, "T8A", challenge, "WT").Cells,
            mode="lines+markers",
            name=challenge,
            showlegend=False,
            line=dict(color=colours[colour]),
        ),
        2,
        2,
    )

# SEPARATE INTO ONE LINE FOR FIRST CONTRACTION AND 3 LINES FOR EXPASIONS


# N3A primary

# Left column
for colour, challenge in enumerate(tetramers):
    fig.add_trace(
        go.Scatter(
            x=times,
            y=fa.separate_data(dataDF, "N3A", challenge, "WT").Cells,
            mode="lines+markers",
            name=challenge,
            showlegend=False,
            line=dict(color=colours[colour]),
        ),
        3,
        1,
    )

# Right column
for colour, challenge in enumerate(tetramers):
    fig.add_trace(
        go.Scatter(
            x=times,
            y=fa.separate_data(dataDF, "N3A", challenge, "WT").Cells,
            mode="lines+markers",
            name=challenge,
            showlegend=False,
            line=dict(color=colours[colour]),
        ),
        3,
        2,
    )

# SEPARATE INTO ONE LINE FOR FIRST CONTRACTION AND 3 LINES FOR EXPASIONS

fig.update_layout(
    updatemenus=[
        dict(
            # type="buttons",  # Remove for dropdown
            direction="down",  # down if dropdown, right if buttons
            x=0.25,
            y=1.06,
            showactive=True,
            xanchor="center",
            yanchor="bottom",
            buttons=list(
                [
                    dict(
                        label="WT",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "WT")}, left_col],
                    ),
                    dict(
                        label="T8A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "T8A")}, left_col],
                    ),
                    dict(
                        label="N3A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "N3A")}, left_col],
                    ),
                    dict(
                        label="WT+T8A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "WT+T8A")}, left_col],
                    ),
                    dict(
                        label="WT+N3A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "WT+N3A")}, left_col],
                    ),
                    dict(
                        label="T8A+N3A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "T8A+N3A")}, left_col],
                    ),
                    dict(
                        label="Triple positive",
                        method="restyle",
                        args=[
                            {"y": fa.data_update(dataDF, "Triple positive")},
                            left_col,
                        ],
                    ),
                    dict(
                        label="Triple negative",
                        method="restyle",
                        args=[
                            {"y": fa.data_update(dataDF, "Triple negative")},
                            left_col,
                        ],
                    ),
                    dict(
                        label="Total",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "Total")}, left_col],
                    ),
                ]
            ),
        ),
        dict(
            direction="down",
            x=0.75,
            y=1.06,
            showactive=True,
            xanchor="center",
            yanchor="bottom",
            buttons=list(
                [
                    dict(
                        label="WT",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "WT")}, right_col],
                    ),
                    dict(
                        label="T8A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "T8A")}, right_col],
                    ),
                    dict(
                        label="N3A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "N3A")}, right_col],
                    ),
                    dict(
                        label="WT+T8A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "WT+T8A")}, right_col],
                    ),
                    dict(
                        label="WT+N3A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "WT+N3A")}, right_col],
                    ),
                    dict(
                        label="T8A+N3A",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "T8A+N3A")}, right_col],
                    ),
                    dict(
                        label="Triple positive",
                        method="restyle",
                        args=[
                            {"y": fa.data_update(dataDF, "Triple positive")},
                            right_col,
                        ],
                    ),
                    dict(
                        label="Triple negative",
                        method="restyle",
                        args=[
                            {"y": fa.data_update(dataDF, "Triple negative")},
                            right_col,
                        ],
                    ),
                    dict(
                        label="Total",
                        method="restyle",
                        args=[{"y": fa.data_update(dataDF, "Total")}, right_col],
                    ),
                ]
            ),
        ),
    ]
)

# fig.show()

fig.write_html("slope-plots.html")

In [61]:
def data_test(positive_data, total_data):
    print(
        f"{'Mouse' : ^9}{'Time' : ^8}{'Positive' : ^12}{'Total' : ^12}{'Fraction' : ^12}{'Correct' : ^11}"
    )
    print("=" * 64)
    for mouse in range(len(positive_data)):
        for current_time in range(len(positive_data[mouse])):
            print(
                f"{mouse + 1 : ^9}{current_time + 1 : ^8}{sum(positive_data[mouse][current_time]) : ^12}{total_data[mouse][current_time][0] : ^12}{round(sum(positive_data[mouse][current_time]) / total_data[mouse][current_time][0], 5) : ^12}{sum(positive_data[mouse][current_time]) <= total_data[mouse][current_time][0] : ^11}"
            )

In [62]:
data_test(n3a_p_data, n3a_p_data_neg)

  Mouse    Time    Positive     Total      Fraction    Correct  
    1       1       84100      4105900     0.02048        1     
    1       2       16790       977210     0.01718        1     
    1       3      9128700     17571300    0.51952        1     
    1       4       125767     5934233     0.02119        1     
    1       5      7558420     20341580    0.37157        1     
    2       1       229240     6920760     0.03312        1     
    2       2       56730      5973270      0.0095        1     
    2       3      16179440    26620560    0.60778        1     
    2       4       141180     4308820     0.03277        1     
    2       5       501620     2138380     0.23458        1     
    3       1       89800      4790200     0.01875        1     
    3       2       64640      7115360     0.00908        1     
    3       3      4755630     7544370     0.63035        1     
    3       4       303210     5166790     0.05868        1     
    3       5      100720

# Tests

In [63]:
test = fa.Mouse(1, 2, 3, 4, 5, 6, 7.257567, 0.124143141)

In [64]:
test.venn()

(1, 2, 4, 3, 5, 6, 7.257567)

In [65]:
round(test)

(1, 2, 4, 3, 5, 6, 7)

In [66]:
test1 = fa.Mouse()
test1.venn()

(-1, 0, 0, 0, 0, 0, 0)

In [67]:
round(test1)

(-1, 0, 0, 0, 0, 0, 0)